In [ ]:
import urllib.request
import requests
import zipfile
import io
import os
import glob
import pandas as pd
import boto
import boto.s3
import sys
import luigi
import datetime, time

import csv
import pandas as pd
import numpy as np
from pandas import DataFrame
import matplotlib.pyplot as plt
import seaborn as sns

from boto.s3.key import Key
from urllib.request import urlopen
from bs4 import BeautifulSoup as bsoup

In [ ]:
def is_file_present(directory,filename):
    if not os.path.exists(directory):
        os.makedirs(directory)
    file_list = glob.glob(directory+'//*.csv')
    for file_name_in_dir in file_list:
        if (directory+ '\\' + filename) == (file_name_in_dir+".zip"):
            return True
    return False

In [ ]:
def download_data(data_type):
    base_URL = "https://resources.lendingclub.com"
    url = urllib.request.urlopen("https://www.lendingclub.com/info/download-data.action")
    content = url.read()
    soup= bsoup(content,'lxml')
    
    #find div by ID
    fileNameDiv = soup.find('div',{"id":data_type})
    FileList = fileNameDiv.text.rstrip("|")

    for fileName in FileList.split("|"):
        file_URL= base_URL+'/'+fileName
        print(file_URL)
        if not is_file_present(data_type,fileName):    
            zfile = requests.get(file_URL)
            z = zipfile.ZipFile(io.BytesIO(zfile.content))
            z.extractall(data_type)

In [ ]:
def read_data(directory):
    fileList = glob.glob(directory+'//*.csv')
    
    dfList=[]
    #columns=["id","member_id","loan_amnt","funded_amnt","funded_amnt_inv","term","int_rate","installment","grade","sub_grade","emp_title","emp_length","home_ownership","annual_inc","verification_status","issue_d","loan_status","pymnt_plan","url","desc","purpose","title","zip_code","addr_state","dti","delinq_2yrs","earliest_cr_line","inq_last_6mths","mths_since_last_delinq","mths_since_last_record","open_acc","pub_rec","revol_bal","revol_util","total_acc","initial_list_status","out_prncp","out_prncp_inv","total_pymnt","total_pymnt_inv","total_rec_prncp","total_rec_int","total_rec_late_fee","recoveries","collection_recovery_fee","last_pymnt_d","last_pymnt_amnt","next_pymnt_d","last_credit_pull_d","collections_12_mths_ex_med","mths_since_last_major_derog","policy_code","application_type","annual_inc_joint","dti_joint","verification_status_joint","acc_now_delinq","tot_coll_amt","tot_cur_bal","open_acc_6m","open_il_6m","open_il_12m","open_il_24m","mths_since_rcnt_il","total_bal_il","il_util","open_rv_12m","open_rv_24m","max_bal_bc","all_util","total_rev_hi_lim","inq_fi","total_cu_tl",
             #"inq_last_12m","acc_open_past_24mths","avg_cur_bal","bc_open_to_buy","bc_util","chargeoff_within_12_mths","delinq_amnt","mo_sin_old_il_acct","mo_sin_old_rev_tl_op","mo_sin_rcnt_rev_tl_op","mo_sin_rcnt_tl","mort_acc","mths_since_recent_bc","mths_since_recent_bc_dlq","mths_since_recent_inq","mths_since_recent_revol_delinq","num_accts_ever_120_pd","num_actv_bc_tl","num_actv_rev_tl","num_bc_sats","num_bc_tl","num_il_tl","num_op_rev_tl","num_rev_accts","num_rev_tl_bal_gt_0","num_sats","num_tl_120dpd_2m","num_tl_30dpd","num_tl_90g_dpd_24m","num_tl_op_past_12m","pct_tl_nvr_dlq","percent_bc_gt_75","pub_rec_bankruptcies","tax_liens","tot_hi_cred_lim","total_bal_ex_mort","total_bc_limit","total_il_high_credit_limit"]
    for filename in fileList:
        print(filename)
        df=pd.read_csv(filename, low_memory=False,skiprows=1)
        print(df.shape)
        ts = time.time()
        df["download_timestamp"] = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
        df["recorded_timestamp"] = filename.rstrip('csv').lstrip('loanStatsFileNamesJS\\').lstrip('rejectedLoanStatsFileNamesJS\\').lstrip('loanStatsFileNamesJS/').lstrip('rejectedLoanStatsFileNamesJS/').lstrip("LoanStats").lstrip("RejectStats").lstrip("_")
        dfList.append(df)
    concatDf=pd.concat(dfList, axis=0)
    #concatDf.columns=columns
    concatDf.to_csv(directory+"_concat_file.csv", index=None)
    print(concatDf.shape)
    return concatDf

In [ ]:
def missing_values_table(df):
       mis_val = df.isnull().sum()
       mis_val_percent = 100 * df.isnull().sum()/len(df)
       mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
       mis_val_table_ren_columns = mis_val_table.rename(
       columns = {0 : 'Missing Values', 1 : '% of Total Values'})
       return mis_val_table_ren_columns

In [ ]:
def getAmazonS3keys():
    #Taking data from user
    access_key = input("Please enter your Amazon S3 Access Key ID:")
    secret_key = input("Please enter your Amazon S3 Secret Access Key:")
    return access_key,secret_key

In [ ]:
def clean_approved_data(loanData):
    
    loanData = loanData[pd.notnull(loanData['loan_amnt'])]
    
    #delete columns which are of no use
    del loanData['emp_title']
    del loanData['desc']
    del loanData['last_pymnt_d']
    del loanData['next_pymnt_d']
    del loanData['last_credit_pull_d']
    
    #Converting to datatime format
    loanData["issue_d"]=pd.to_datetime(loanData['issue_d'])
    loanData["earliest_cr_line"]=pd.to_datetime(loanData['earliest_cr_line'])
    #adding Issue_Month and Issue_Year columns
    loanData['issue_month'] = loanData['issue_d'].dt.month
    loanData['issue_year'] = loanData['issue_d'].dt.year

    #Deriving Credit age for every customer
    loanData["credit_age"]=loanData["issue_d"]-loanData["earliest_cr_line"]

    #Derivinf Lending club interest rate
    loanData["num_il_tl"]=loanData["num_il_tl"].fillna(0)
    loanData["num_rev_accts"]=loanData["num_rev_accts"].fillna(0)
    loanData["Lending_Club_Interest"]=loanData["total_pymnt"]-loanData["total_pymnt_inv"]
    
    #Format Interest_Rate
    loanData["Interest_Rate"]=loanData["int_rate"].str.rstrip("%")
    loanData['Interest_Rate'] = loanData['Interest_Rate'].convert_objects(convert_numeric=True)
    
    #Modifying emp_length
    loanData["derived_emp_length"]=loanData["emp_length"]
    loanData.derived_emp_length[loanData.derived_emp_length=="10+ years"]="10"
    loanData.derived_emp_length[loanData.derived_emp_length=="< 1 year"]="0"
    loanData['derived_emp_length'] = loanData.derived_emp_length.str.replace(' years?' , '')

    #Modifying title by personal
    loanData['derived_title'] = loanData['title']
    loanData['derived_title'] = loanData['derived_title'].fillna('personal')
    
    #mths_since_last_delinq - replace by 24 if delinq_2yrs = 0
    loanData['derived_mths_since_last_delinq'] = loanData['mths_since_last_delinq']
    loanData['derived_mths_since_last_delinq'] = loanData['derived_mths_since_last_delinq'].fillna(1000)
    
    b1 = loanData.id.where(loanData.delinq_2yrs == 0)
    b2 = loanData.id.where(loanData.derived_mths_since_last_delinq == 1000)
    b3 = set(b1).intersection(b2)
    for i in b3:
        for j in loanData.id:
            if i==j:
                loanData.derived_mths_since_last_delinq[loanData.id == j] = 24
    
    #mths_since_last_record - replace blanks by -1, which indicates that a borrower doesn't have any public record
    loanData['derived_mths_since_last_record'] = loanData['mths_since_last_record']
    loanData['derived_mths_since_last_record'] = loanData['derived_mths_since_last_record'].fillna(-1)
    
    #annual_inc - there are 4 blank records, each doesn't have a employer so replacing it with zero
    loanData['derived_annual_inc'] = loanData['annual_inc']
    loanData['derived_annual_inc'] = loanData['derived_annual_inc'].fillna(0)
    
    #delinq_2yrs - delinq_2yrs has the max value of 39. Whereas the delinq_2yrs is null for all people which does not meet the
    # credit policy is not valid and who status is Not Verified. So we are taking the mean of all the 
    loanData['derived_delinq_2yrs'] = loanData['delinq_2yrs']
    loanData['delinq_2yrs'] = loanData['delinq_2yrs'].fillna(100)
    
    #delinq_amnt - Replacing this by 0, because 99.98% of the values is zero. Negligible rows has some other values
    loanData['derived_delinq_amnt'] = loanData['delinq_amnt']
    loanData['derived_delinq_amnt'] = loanData['derived_delinq_amnt'].fillna(0)
    
    #revol_util - Replaced it with the mean
    loanData['derived_revol_util'] = loanData['revol_util']
    loanData["derived_revol_util"] = loanData["derived_revol_util"].str.rstrip("%")
    loanData['derived_revol_util'] = loanData['derived_revol_util'].convert_objects(convert_numeric=True)
    loanData['derived_revol_util'] = loanData['derived_revol_util'].fillna((loanData['derived_revol_util'].mean()))

    #pub_rec_bankruptcies - Replace this by the mean as all are non verified and does not meet the credit policy.
    loanData['derived_pub_rec_bankruptcies'] = loanData['pub_rec_bankruptcies']
    loanData['derived_pub_rec_bankruptcies'] = loanData['derived_pub_rec_bankruptcies'].fillna(loanData['derived_pub_rec_bankruptcies'].mean())

    #tax_liens
    loanData['derived_tax_liens'] = loanData['tax_liens']
    loanData['derived_tax_liens'] = loanData['derived_tax_liens'].fillna(loanData['derived_tax_liens'].mean())

    #interest_rate
    loanData['derived_int_rate'] = loanData['int_rate']
    loanData["derived_int_rate"] = loanData["derived_int_rate"].str.rstrip("%")
    loanData['derived_int_rate'] = loanData['derived_int_rate'].convert_objects(convert_numeric=True)
    
    #FICO
    loanData["FICO"]=(loanData["last_fico_range_high"]+loanData["last_fico_range_low"])/2
    loanData["FICO"] = loanData[['FICO']].convert_objects(convert_numeric=True).fillna(300)
    
    #Total_Acc
    loanData["total_acc"] = loanData[['total_acc']].convert_objects(convert_numeric=True).fillna(loanData["total_acc"].mean())
    
    #Account open past 24 months
    loanData["acc_open_past_24mths"] = loanData[['acc_open_past_24mths']].convert_objects(convert_numeric=True).fillna(loanData["acc_open_past_24mths"].mean())
    
    #Taking only numeric values from term
    loanData["Derived_term"]=loanData["term"].str.extract("(\d+) months",expand=True)
    loanData["Derived_term"] = loanData[['Derived_term']].convert_objects(convert_numeric=True)
    
    return loanData
    #loanData.to_csv("Approved_Loans_clean_file.csv", sep=",")

In [ ]:
def analysize_approved_data(data):

    import csv
    import pandas as pd
    import numpy as np
    from pandas import DataFrame
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    #Analysis 1: Month-wise count of number of loans issued
    result1=data.groupby(["issue_month"])["id"].count().reset_index(name="Count").sort_values(["issue_month"])
    result1.plot(x="issue_month",y="Count",kind="bar",title="Month-wise count of number of loans issued")
    plt.show()
    
    #Analysis 2: State-wise count of number of loans issued
    result2=data.groupby("addr_state")["id"].count().reset_index(name="Count").sort_values("Count",ascending=True)
    result2.plot(x="addr_state",y="Count",kind="bar",title="State-wise count of number of loans issued")
    plt.show()
    
    #Analysis : Month wise number of loans issued in CA
    x1=data[data["addr_state"]=="CA"]
    x2=x1.groupby(["issue_month"])["id"].count().reset_index(name="Count").sort_values(["issue_month"])
    x2.plot(x="issue_month",y="Count",kind="bar",title="Month wise number of loans issued in CA")
    plt.show()
    
    #Analysis 3: Frequency for every grade and sub-grade
    result3=data.groupby(["grade","sub_grade"])["id"].count().reset_index(name="Count")
    result3.plot(x="sub_grade",y="Count",kind="line",title="Title3")
    plt.show()
    
    #Analysis 4: Loan purpose for all members
    result4=data.groupby("purpose")["id"].count().reset_index(name="Count").sort_values("Count",ascending=False)
    result4.plot(x="purpose",y="Count",kind="pie",title="Title4")
    plt.show()
    
    #Analysis 5: Mean Interest Rate for every Verification  status
    g1 = sns.barplot(x="issue_year", y="Interest_Rate", hue="verification_status", data=data)
    sns.plt.show()
    
    #Analysis 6: House ownership status for all members
    g = sns.barplot(x="issue_year", y="Interest_Rate", hue="home_ownership", data=data)
    sns.plt.show()

In [ ]:
loan_data = 'loanStatsFileNamesJS'
download_data(loan_data)
loanData = read_data(loan_data)
loanData.to_csv("Approved_Loans_Combined_Data.csv", sep=",")

In [ ]:
loanData = pd.read_csv("Approved_Loans_Combined_Data.csv", encoding= 'iso-8859-1',low_memory=False)
print('Missing values in loan approved data')
print(missing_values_table(loanData))
loanData = clean_approved_data(loanData)
#Write clean data to new CSV
loanData.to_csv("Cleaned_Approved_Loans_Combined_Data.csv", sep=",")

In [ ]:
loanData = pd.read_csv("Cleaned_Approved_Loans_Combined_Data.csv", encoding= 'iso-8859-1',low_memory=False)
analysize_approved_data(loanData)

In [ ]:
AWS_ACCESS_KEY_ID = input("Please enter AWS access key")
AWS_SECRET_ACCESS_KEY = input("Please enter AWS secret key")

bucket_name = 'lending-club-analysis'

conn = boto.connect_s3(AWS_ACCESS_KEY_ID,
        AWS_SECRET_ACCESS_KEY)

bucket = conn.create_bucket(bucket_name,
    location=boto.s3.connection.Location.DEFAULT)

   
file = "Cleaned_Approved_loans_combinedData.csv"
print ('Uploading %s to Amazon S3 bucket %s' % \
   (file, bucket_name))

def percent_cb(complete, total):
    sys.stdout.write('.')
    sys.stdout.flush()

k = Key(bucket)
k.key = file
k.set_contents_from_filename(file,
    cb=percent_cb, num_cb=10)